# Coordinate descent: correction
For any enquiry about this lab, mail pierre.ablin@inria.fr .

## Code a function that computes $g(x)$ and $\text{prox}_g(x)$ for both cases

### Correction

In [ ]:
def l1(x, l):
    return np.sum(np.abs(x)) * l

def indicator(x, l):
    return np.inf if np.any(x < 0) else 0.

def prox_l1(x, l):
    return np.sign(x) * np.maximum(np.abs(x) - l, 0.)

def prox_indicator(x, l):
    return np.maximum(x, 0)

### Comments
Vectorize as much as possible! Lists should not be involved in these functions. Note that using `map` does not really vectorize, it is just a shortcut for a loop. 

The indicator and prox indicator should not depend on the regularization parameter.

You can return $+\infty$ in Python, the best way to do so is `np.inf`, and not `math.inf` or `float('inf')`. 

## Justify why proximal coordinate descent can be applied to obtain a minimum of such objective functions.

### Correction
The objective function is the sum of a smooth convex function and separable convex functions.

## Code the proximal gradient descent
### Correction

In [ ]:
def cd_linreg(x0, A, b, g, prox_g, s=0., n_iter=50,
              x_true=coefs, verbose=True):
    x = x0.copy()
    L = np.sum(A * A, axis=0) # Precompute the Lipschitz constants
    residual = b - np.dot(A, x)
    for k in range(n_iter + 1):
        i = k % n_features
        xi_old = x[i].copy()  # Used for the smart residual
        A_i = A[:, i]
        gradient_i = -np.dot(A_i, residual) # Gradient formula. Doing np.dot(A_i.T, residual) is useless ! 
        x[i] -= 1. / L[i] * gradient_i
        x[i] = prox_g(x[i], s / L[i])
        residual -= A_i * (x[i] - xi_old)  # Smart update: no need to recompute b - np.dot(A, x)

### Comments

Precompute the Lipschitz constants, once and for all before the iterations. Computing a Lipschitz constant per iteration is inefficient.

The smart residual has to be derived and coded. Say that at some iteration, the $i$-th feature goes from $x[i]_{old}$ to $x[i]$. Then, the residual $r = b - Ax$ becomes $r - A[:,i](x[i] - x[i]_{old})$.

Transposing a numpy 1-D array does nothing. For instance, `A[:, i].T` is the same thing as `A[:, i]`. So when you compute the gradient, no need to do `np.dot(A[:, i].T, residual)`.

Do not forget to add the step size in the regularization parameter of the prox!

## Evaluate the convergence depending on the conditioning

The convergence should be impaired by high conditioning. 

For convergence plot: when you monitor a quantity that goes to 0, a logarithmic plot is generally clearer. It's much easier to see if the convergence is quadratic or linear for instance. To do so, use the function `plt.semilogy(x, y)` rather than `plt.plot(x, np.log10(y))` as the y-axis will automatically be log scaled. 

You should also avoid log-log plots, unless two algorithms have orders of magnitude of difference in their convergence speed.

The plots should have labeled axis, titles, and line legends when there is more than one line.

Do not plot too many things on the same figure. $>5$ curves is too much.

In order to have more readable, maintainable and beautiful code, when trying different parameters, you should use loops. 
Something like this:

In [ ]:
A, b = simu_linreg(coefs,corr=0.1)
x_1,objectives_1,errors_1 = cd_linreg(np.zeros(len(coefs)),A,b , lasso , prox_lasso,n_iter = 1000,verbose=False)

#Correlation : 0.5
A, b = simu_linreg(coefs,corr=0.5)
x_2,objectives_2,errors_2 = cd_linreg(np.zeros(len(coefs)),A,b , lasso , prox_lasso,n_iter = 1000,verbose=False)

#Correlation : 0.9
A, b = simu_linreg(coefs,corr=0.9)
x_3,objectives_3,errors_3 = cd_linreg(np.zeros(len(coefs)),A,b , lasso , prox_lasso,n_iter = 1000,verbose=False)

plt.plot(range(n_iter + 2),errors_1, color = 'blue' , label = 'Correlation : 0.1')
plt.plot(range(n_iter + 2),errors_2, color = 'red',  label = 'Correlation : 0.5')
plt.plot(range(n_iter + 2),errors_3, color = 'green', label = 'Correlation : 0.9')
plt.legend()
plt.show()

Can be compactly rewritten as:

In [ ]:
corr_list = [0.1, 0.5, 0.9]
colors = ['blue', 'red', 'green']
for corr, color in zip(corr_list, colors):  # zip allows to iterate on several lists at a time.
    A, b = simu_linreg(coefs, corr=corr)
    x, objective, errors = cd_linreg(np.zeros(len(coefs)), A, b, lasso, prox_lasso, n_iter=1000, verbose=False)
    label = 'Correlation : %.2g' % corr  # If corr = 0.1, this is the string 'Correlation : 0.1'.
    plt.semilogy(errors, color=color, label=label)  # Log plot please 
plt.legend()
plt.show()

It does not really make sense to compare the convergence error of the problem penalized with lasso and penalized with the indicator. These are two entirely different problems. For instance, when investigating the effect of correlation on convergence speed, you should do one plot per penalization with different convergence curves, rather than one plot per correlation with the two different penalizations.


When you want to plot the convergence curves of an algorithm over `n_iter` iterations, you should first run in a larger number of times (i.e. `5 * n_iter` times) to compute accurately the minimum and minimizer of the loss. If you just take the last iterate as the minimizer, you will observe a dubious dip in the error which is an artifact.

The error **should go to zero**, otherwise it means that you've taken a wrong `x_true`, that your code is not correct, or that you have not given the algorithm enough iterations to converge. The same thing goes for the distance to the minimum. The whole problem that you are solving can be written as:
$$\text{minimize } f(x) + \lambda g(x)$$
What we call minimum/minimizer is the minimum/minimizer of $f + \lambda g$, not of only $f$!

In all the cases studied before, there is a unique minimum. So if everything goes as planned, each optimization algorithm in the world should converge to that minimum. Comments about "algo A has a slightly smaller error than algo B" do not make sense, the error should simply go to 0! The whole point of this lab is to observe the behavior of the algorithms. The convergence point does not depend on the algorithm, thus talking about it is irrelevant.

This is why the value of `x_true` should be computed beforehand by running the algorithm for a large number of iterations. 

As a summary: on each plot, the error and distance to the minimum curves should go to 0, and thus should be plotted in log-scale.

If you want to plot the convergence curve of Fista and CD for different correlations, you have to maintain consistency in the colors / linestyle. For instance, plot each curve for Fista using dotted lines, each curve for CD in full lines, and choose one color per correlation. It will make things much clearer. 

## Compare with prox gradient descent

In this question, it is key to be specific about what the number of iterations mean. One iteration of coordinate descent can mean two things: either a full pass on the whole feature set, or just one coordinate update. As seen in class, with the smart residual update and the precomputed Lipschitz constants, the complexity of a pass on the full data with coordinate descent is similar to the complexity of one iteration of gradient descent. It is thus much fairer to compare the **number of passes on the full data** with coordinate gradient descent to the number of iterations of gradient descent.

To do so, the `n_iter` parameter of `cd_linreg` should be `n_features` times the one of `fista`/ `ista`, and to have a fair visualization you could for instance do:

In [ ]:
plt.semilogy(np.linspace(0, n_iter, num=n_features * n_iter), error_cd)  # Here n_iter is the number of iterations of ista/fista
plt.semilogy(error_gradient_descent)

Although plotting the error vs time might seem like a good idea, in this lab nothing is done to enforce the fairness of such a comparison. The results that you are going to obtain are driven by the efficiency your implementation rather than by the underlying algorithm. Those of you who tried generally observed that CD was outperformed by gradient descent, which would not be true with a very efficient implementation of the algorithms.

If you want to compare how conditioning affects the behavior of the two algorithms, you should either plot the curves for different correlations on the same graph, or at the very least have fixed xlimits and y limits for the plots. Otherwise, it makes comparison tedious.

# Misc.
Please upload notebooks that you have already run, where the figures look nice etc... Especially if the notebook takes a long time to compile!

Please turn the verbose off if you run your algorithm for more than 100 iterations. 